
## Глава 3 Теоремы теории вероятностей

**Кандидат должен знать:**

- Закон больших чисел и усиленный закон больших чисел
- Центральная предельная теорема
- Многомерные распределения

**Материалы:**
- [Райгородский курс по тер. веру лекции 8,9,12,13](https://www.youtube.com/watch?v=n7jx2MfIeX4&list=PLthfp5exSWEr8tRK-Yf-i9aXgcFJ-O16d&index=8)
- [курс ТВ  на степике из частей 2,4](https://stepik.org/course/3089)


## Задачи с кодингом

### 1. Normal Random Variable Generation

Let us have a random variable $\xi \sim U[0,1]$.

**Task:**
1. Based on this random variable, use the Central Limit Theorem (CLT) to simulate ($n = 10000$) random variable $\eta \sim N(\mu, \sigma^2)$.

2. Plot the density distribution and compare it with the theoretical distribution.

3. Calculate mean and variance, and compare them with the true values.

**Notes**

1. To generate $\xi$, you can use numpy function `numpy.random.random`
2. To plot theoretical density distribution of a normal variable - `scipy.stats.norm.ppf`


### 1. Normal Random Variable Generation

Let $X$ be a uniformly distributed random variable, and $\eta$ be a normally distributed random variable.

$\eta = \frac{1}{N_x} \sum_{i=1}^{N_x} X_i$
<br>
$\mu_{\eta} = \frac{1}{N_x} E \left( \sum_{i=1}^{N_x} X_i \right) = \frac{1}{N_x} \sum_{i=1}^{N} E_{X_i} = E_X$
<br>

$
\sigma_\eta^2 = \frac{1}{(N_x-1)^2} \sum_{i=1}^{N_x} D_{X_i} = \frac{N_x \cdot D_X}{(N_x-1)^2} \approx \left[\text{if } N_x > 100\right] \approx \frac{D_X}{N_x}
$


<br><br><br>
$E_X = \mu_{\eta}$
<br>
$D_X = N_x \cdot \sigma_\eta^2$

Now I need random uniform values $X$ with $\mu_X$ and $\sigma_x^2$:


\begin{cases}
E_X = \frac{a + b}{2} \\
D = \frac{(b - a)^2}{12} 
\end{cases}

\begin{cases}
a = E_X - \sqrt{3 \cdot D_X} \\
b = E_X + \sqrt{3 \cdot D_X}
\end{cases}


$X = a + Z \cdot (b - a) \quad \text{where} \quad Z \sim U[0, 1]$

In [1]:
import math
from math import comb
from collections import Counter

import numpy as np
from scipy.stats import norm, binom

import plotly.express as px
import plotly.graph_objects as go

In [2]:
# images from task_3/plots will be used if False
MAKE_PLOTS = False

In [3]:
def generate_eta_1(mu_eta, sigma_squared_eta, n_x, n_eta):
    expectation_x = mu_eta
    var_x = n_x * sigma_squared_eta

    a = expectation_x - np.sqrt(3 * var_x)
    b = expectation_x + np.sqrt(3 * var_x)

    Z = np.random.random((n_eta, n_x))
    X = a + Z * (b - a)

    return X.mean(axis=1)

### Or another way:

To generate $\eta \sim N(\mu, \sigma^2)$:
1. Generate $X \sim U(0, 1)$ and sum $n_x$ samples to get `sum_uniforms` with the shape of the number of samples $\eta$.
2. Standardize the `sum_uniforms` to get standard normal values. $E_X = n_x / 2$, $D_X = n_x / 12$.
3. Scale and shift using parameters $N(\mu, \sigma^2)$ to get the final normal samples $\eta$.



In [4]:
def generate_eta_2(mu_eta, sigma_squared_eta, n_x, n_eta):
    uniform_random = np.random.random((n_eta, n_x))
    sum_uniforms = np.sum(uniform_random, axis=1)
    standard_normal = (sum_uniforms - n_x / 2) / np.sqrt(n_x / 12)
    return mu_eta + np.sqrt(sigma_squared_eta) * standard_normal

### Expectation and Variance Calculation

In [5]:
# choose generation algorithm: generate_eta_1 or generate_eta_2
generate_eta = generate_eta_2

expectation_eta = -7
var_eta = 4
sigma_eta = np.sqrt(var_eta)
number_of_x = 1000
number_of_eta = 10**5

eta = generate_eta(expectation_eta, var_eta, number_of_x, number_of_eta)

print("###  Task 3: Theoretical E and D VS Simulated E and D  ###\n")
print(f"Theoretical E = {expectation_eta}; Simulated E = {eta.mean()}")
print(f"Theoretical D = {var_eta}; Simulated D = {eta.var()}")

###  Task 3: Theoretical E and D VS Simulated E and D  ###

Theoretical E = -7; Simulated E = -7.0046769097708514
Theoretical D = 4; Simulated D = 4.04253475104238


In [6]:
fig = px.histogram(
    eta,
    nbins=40,
    title="Distribution of eta",
    labels={"value": "Value"},
    height=700,
    width=500,
)

In [7]:
if MAKE_PLOTS:
    fig.show()

![](plots/norm_distribution.png)

### Density Plot. Comparing with Theoretical Plot

In [8]:
fig = go.Figure()

norm_x = np.linspace(
    norm.ppf(0.0001, loc=expectation_eta, scale=sigma_eta),
    norm.ppf(0.9999, loc=expectation_eta, scale=sigma_eta),
    number_of_eta,
)
norm_y = norm.pdf(norm_x, loc=expectation_eta, scale=sigma_eta)

fig.add_trace(
    go.Scatter(
        x=norm_x,
        y=norm_y,
        mode="lines",
        line=dict(color="darkblue", width=5),
        name="norm pdf",
    )
)

hist_y, hist_x = np.histogram(eta, bins="auto", density=True)
fig.add_trace(
    go.Bar(
        x=hist_x,
        y=hist_y,
        opacity=0.8,
        name="histogram",
        marker=dict(color="darkgreen"),
    )
)

fig = fig.update_layout(
    title="Normal Distribution",
    xaxis_title="Value",
    yaxis_title="Density",
    template="plotly_dark",
    legend=dict(x=0.8, y=1.0),
    height=700,
    width=500,
)

In [9]:
if MAKE_PLOTS:
    fig.show()

![](plots/norm_pdf.png)

### 2. Casino 

There are two games:

**a. Dice:**

Player pays 4&#36; and rolls two dice. If sum 8 or more, player gets a prize equal to the sum.

**b. Разгадай код:**

There is a five-digit code (each digit from 0 to 9). Player pays 10&#36; and tries to guess the code:
    
    - Player guesses 0 or 1 digit: gets nothing
    - Player guesses exactly 2 digits: gets 50
    - Player guesses exactly 3 digits: gets 200
    - Player guesses exactly 4 digits: gets 5000
    - Player guesses the whole code: gets 100000
    After each try, the code changes.

**Task:**

Let $\xi$ and $\eta$ be a random variables equal to the player's win/loss in the dice and code games, in one round.

1. Calculate $E\xi$ and $E\eta$
2. Simulate random variables $\xi$ and $\eta$
3. Plot the average win/loss convergence to the expectation
4. Compare the convergence speed of the random variables to their expectations. Why are they different?
5. How many experiments would you take to estimate the expectation?

## a. Dice
### 1) Expectation Calculation

In [10]:
def calc_expectation_and_var(game_probs, game_values):
    expectation = sum([prob * val for prob, val in zip(game_probs, game_values)])
    second_moment = sum([prob * val**2 for prob, val in zip(game_probs, game_values)])

    var = second_moment - expectation**2
    return expectation, var

In [11]:
N_DICE_SIDES = 6

one_game_cost = 4
win_threshold = 8

win_amounts = (
    np.array([0 if i < win_threshold else i for i in range(2 * N_DICE_SIDES + 1)])
    - one_game_cost
)
possible_dice_values = [
    i + j for i in range(1, N_DICE_SIDES + 1) for j in range(1, N_DICE_SIDES + 1)
]

count_values = Counter(possible_dice_values)
probs = [
    count_values[dice_value] / N_DICE_SIDES**2
    for dice_value in range(2 * N_DICE_SIDES + 1)
]
value_to_prob_str = ",  ".join(
    [f"{i}: {count_values[i]}/{N_DICE_SIDES ** 2}" for i in range(2 * N_DICE_SIDES + 1)]
)

print("Value: Probability -", value_to_prob_str)
print("Probs sum:", sum(count_values.values()) / N_DICE_SIDES**2)

Value: Probability - 0: 0/36,  1: 0/36,  2: 1/36,  3: 2/36,  4: 3/36,  5: 4/36,  6: 5/36,  7: 6/36,  8: 5/36,  9: 4/36,  10: 3/36,  11: 2/36,  12: 1/36
Probs sum: 1.0


In [12]:
expectation_xi, var_xi = calc_expectation_and_var(probs, win_amounts)

print("Theoretical E_xi:", round(expectation_xi, 5))
print("Theoretical D_xi:", round(var_xi, 5))

Theoretical E_xi: -0.11111
Theoretical D_xi: 21.82099


### 2) Random Variable Simulation

In [13]:
def generate_xi(n_samples, win_values):
    dice_values = np.random.randint(1, N_DICE_SIDES + 1, 2 * n_samples)
    dice_sums = dice_values[::2] + dice_values[1::2]
    return win_values[dice_sums]

In [14]:
np.random.seed(48)
n_games = 9 * 10**3

xi_values = generate_xi(n_games, win_amounts)

print("Theoretical E xi =", round(expectation_xi, 5))
print("Simulated E xi =", xi_values.mean())

Theoretical E xi = -0.11111
Simulated E xi = -0.11477777777777778


### 3. Average Win Convergence to Expectation

In [15]:
def plot(x_data, mean_value):
    cumulative_means = np.cumsum(x_data) / np.arange(1, len(x_data) + 1)

    fig = go.Figure()

    fig.add_trace(
        go.Scatter(
            x=np.arange(1, len(x_data) + 1),
            y=cumulative_means,
            mode="lines",
            name="Average win value",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=[1, len(x_data)],
            y=[mean_value, mean_value],
            mode="lines",
            line=dict(dash="dash", color="red"),
            name="Expected value",
        )
    )

    fig.update_layout(
        title="Average Win Convergence to Expectation",
        xaxis_title="Number of Games",
        yaxis_title="Win Amount",
        legend=dict(x=0.8, y=1.0),
        template="plotly_dark",
    )

    fig.show()

In [16]:
if MAKE_PLOTS:
    plot(xi_values, expectation_xi)

![](plots/dice_expectation.png)

### 5) Number of Experiments Estimation

According to the Central Limit Theorem, the sample mean $\bar{X}_n$ of a sample size $n$ from a population with mean $\mu$ and variance $\sigma^2$ follows a normal distribution:

$$\bar{X}_n \sim N\left(\mu, \frac{\sigma^2}{n}\right)$$

I need to ensure that the probability that $\bar{X}_n$ falls within the interval [$\mu - \epsilon$, $\mu + \epsilon$] is equal to $p$:
   $$P\left(|\bar{X}_n - \mu| < \epsilon\right) = p$$

1. Standardizing: by subtracting the mean and dividing by the standard deviation:
   $$P\left(|\bar{X}_n - \mu| < \epsilon\right) = P\left(\left| \frac{\bar{X}_n - \mu}{\frac{\sigma}{\sqrt{n}}} \right| < \frac{\epsilon}{\frac{\sigma}{\sqrt{n}}}\right) = P\left(|Z| < \frac{\epsilon \sqrt{n}}{\sigma}\right) = p$$

2. For $p = 0.95$:
   $$\frac{\epsilon \sqrt{n}}{\sigma} = z = 1.96$$ 

3. Finding the minimum number of experiments $n$ required to achieve the specified accuracy:
   $$n \geq \left( \frac{z \sigma}{\epsilon} \right)^2$$


In [17]:
def estimate_n(variance_x, eps_x, x):
    return ((x * np.sqrt(variance_x)) / eps_x) ** 2

In [18]:
z = 1.96
eps = 0.05

n = int(estimate_n(var_xi, eps, z))

print("For 95% confidence level, and 5% error, n should be at least", n)

For 95% confidence level, and 5% error, n should be at least 33531


## b. Guess the Code
### 1)  Expectation Calculation

In [19]:
GUESS_DIGIT_PROB = 1 / 10  # probability of guessing one digit
CODE_LENGTH = 5

one_game_cost = 10

guess_code_win_amounts = np.array([0, 0, 50, 200, 5000, 100000]) - one_game_cost
number_of_successes = np.arange(CODE_LENGTH + 1)
code_game_probs = binom.pmf(number_of_successes, CODE_LENGTH, GUESS_DIGIT_PROB)

print("Probs:", code_game_probs)

Probs: [5.9049e-01 3.2805e-01 7.2900e-02 8.1000e-03 4.5000e-04 1.0000e-05]


In [20]:
expectation_eta, var_eta = calc_expectation_and_var(
    code_game_probs, guess_code_win_amounts
)

print("Theoretical E eta =", round(expectation_eta, 5))
print("Theoretical D eta =", round(var_eta, 5))

Theoretical E eta = -1.485
Theoretical D eta = 111683.74478


### 2) Random Variable Simulation

In [21]:
def generate_eta(n_samples, win_values):
    random_values_code = np.random.randint(0, 10, (n_samples, CODE_LENGTH))
    code_guesses = np.random.randint(0, 10, (n_samples, CODE_LENGTH))

    is_guess_random_code = code_guesses == random_values_code
    random_guesses = is_guess_random_code.sum(axis=1)
    return win_values[random_guesses]

In [22]:
np.random.seed(12)
n_games = 1 * 10**6

eta_values = generate_eta(n_games, guess_code_win_amounts)

print("Theoretical E eta =", round(expectation_eta, 5))
print("Simulated E eta =", eta_values.mean())

Theoretical E eta = -1.485
Simulated E eta = -1.4936


### 3. Average Win Convergence to Expectation

In [23]:
if MAKE_PLOTS:
    plot(eta_values, expectation_eta)

![](plots/code_expectation.png)

### 4) Convergence Speeds Comparing

Convergence speed depends on the variance of the random variable. 

Dice game has a smaller variance, so the convergence is faster.

Guess the code game has a larger variance, so the convergence is slower.

In [24]:
print("D xi:", var_xi)
print("D eta:", var_eta)

D xi: 21.82098765432099
D eta: 111683.74477500001


### 5) Number of Experiments Estimation

In [25]:
n = int(estimate_n(var_eta, eps, z))

print("For 95% confidence level, and 5% error, n should be at least", n)

For 95% confidence level, and 5% error, n should be at least 171617709


### Random Graph

Let $G(n,p)$ be a random graph on $n$ vertices, where $p$ - the probability of an edge between any two nodes.

And $\xi(n,p)$ - the number of triangles (complete subgraphs on three vertices) in the random graph.

**Task 1:**

1. Find $E\xi$.
2. Implement a simulation of a random graph and a function to find the number of triangles.
3. $G(3,p)$ - random graph on three vertices. Implement $\tau$ - random variable equal to 1 if the graph is complete (forms a triangle) and 0 otherwise.

**Task 2:**

- Let $M(k)$ be the average number of triangles after $k$ generations of a random graph.
- Let $T(k)$ be the average number of triangles after generating a random graph on three vertices (generating $C_n^3$ graphs on three vertices in each iteration).

1. Is it true that $M(k) \rightarrow E\xi$ as $k \rightarrow \infty$?
2. Is it true that $T(k) \rightarrow E\xi$ as $k \rightarrow \infty$?

3. Consider two graphs $G_1(10, \frac{1}{2})$ and $G_2(10, \frac{1}{20})$. Plot $M(k)$, $T(k)$, and $E\xi$ (constant) for this graphs.

**Task 3:**

Perform similar task for $D\xi$:
- Find $D\xi$ mathematically.
- Calculate the variance of the number of triangles by simulating the random graph $k$ times.
- Calculate the variance of the number of triangles by generating $C_n^3$ realizations of the random variable $\tau$, $k$ times.
- Estimate the convergence.
- Plot graphs for $G_1(10, \frac{1}{2})$ and $G_2(10, \frac{1}{20})$.


## Task 1
#### Random Graph on n Vertices. Find $E\xi$

Let $I$ be a random variable equal to 1 if a triangle is formed between three specific nodes, else 0. Then $\xi = \sum_{i=1}^{C_n^3} I_i$.

$E\xi = E\sum_{i=1}^{C_n^3} I_i = \sum_{i=1}^{C_n^3} E I_i = C_n^3 \cdot p^3$

#### G(3,p) - Random Graph on Three Vertices.

For one graph: $E\tau_1 = 1 \cdot p^3 + 0 \cdot (1 - p^3) = p^3$

And I generate $C_n^3$ graphs on each iteration: $T(k) = C_n^3 \cdot E\tau_1 = C_n^3 \cdot p^3$

#### Random Graph Simulation and Function for Finding the Number of Triangles

I will use an adjacency matrix, filled only above the main diagonal, as a graph.

The square of the adjacency matrix contains the number of paths of length 2 between vertices. By subsequent element-wise multiplication of the matrix I will find the number of triangles in the graph. Since I use only the upper triangular adjacency matrix, duplicates in the triangle count do not appear.

Elements in squared adjacency matrix can take values from 0 to $n - 1$, where $n$ - the number of nodes in the graph. Therefore, to use memory efficiently, int8 dtype can be used when $n \leq 128$, and int16 when $128 < n < 32768$.

In [26]:
def generate_xi_graph(k_graphs, n_nodes, edge_p):
    """Generate random graphs on n_nodes vertices with edge probability edge_p"""
    MAX_NODES = 3 * 10**3
    if n_nodes > MAX_NODES:
        raise ValueError(
            f"Too many nodes. It will take too much time to process {n_nodes} nodes. Use less than {MAX_NODES} nodes."
        )

    dtype = np.int8 if n_nodes <= np.iinfo(np.int8).max else np.int16
    bool_values = np.random.random((k_graphs, n_nodes, n_nodes)) < edge_p
    int_values = bool_values.astype(dtype)

    adjacency_matrices = np.triu(int_values, k=1)
    return adjacency_matrices


def count_triangles_in_xi_graph(graphs):
    """Get xi random variable - the number of triangles in each graph"""
    squared_matrix = np.matmul(graphs, graphs)
    triangles_matrix = graphs * squared_matrix
    return np.sum(np.triu(triangles_matrix, k=1), axis=(1, 2))

In [27]:
def generate_xi(k_graphs, n_nodes, edge_p):
    """Generate xi random variable - the number of triangles in the graph"""
    graphs = generate_xi_graph(k_graphs, n_nodes, edge_p)
    return count_triangles_in_xi_graph(graphs)


def generate_tau(k_graphs, n_nodes, edge_p):
    """Generate tau random variable - the number of triangles in the graph on three vertices, generated C_n^3 times"""
    shape = (k_graphs, math.comb(n_nodes, 3))
    tau_graph = (np.random.random(shape) < edge_p**3).astype(np.int8)
    return tau_graph.sum(axis=1)

## Task 2

In [28]:
k = 10**5
n = 10
p = 0.5

print("E xi = E tau =", math.comb(n, 3) * p**3)

E xi = E tau = 15.0


#### Random Graph on n Vertices. M(k):

In [29]:
xi = generate_xi(k, n, p)
print("M_k:", np.mean(xi))

M_k: 15.00141


#### G(3,p) - Random Graph on Three Vertices. T(k):

In [30]:
tau = generate_tau(k, n, p)
print("T_k:", np.mean(tau))

T_k: 15.00769


### Convergence of Expectation (Law of Large Numbers)

#### From the linearity of the mathematical expectation:

$E[\bar{X}] = E\left[\frac{1}{n}\sum_{i=1}^n X_i\right] = \frac{1}{n} \sum_{i=1}^n E[X_i] = \frac{1}{n} \cdot n \cdot \mu = \mu$
<br>

#### The Weak Law of Large Numbers (Convergence in Probability):
$\lim_{n \to \infty} P(|\bar{X} - \mu| \geq \epsilon) = 0$, для любого $\epsilon > 0$

$M(k) \rightarrow E\xi$ при $k \rightarrow \infty$ <br>
$T(k) \rightarrow E\xi$ при $k \rightarrow \infty$

Both statements are true

Consider $G_1(10, \frac{1}{2})$ and $G_2(10, \frac{1}{20})$. And plot $M(k)$, $T(k)$, and $E\xi$


In [31]:
def get_xi_and_tau_and_theoretical_expectations(n_graphs, n_nodes, edge_p):
    theoretical_expectation = math.comb(n_nodes, 3) * edge_p**3

    xi_graph = generate_xi_graph(n_graphs, n_nodes, edge_p)
    xi_variable = count_triangles_in_xi_graph(xi_graph)

    tau_variable = generate_tau(n_graphs, n_nodes, edge_p)

    xi_expectations = np.cumsum(xi_variable) / np.arange(1, n_graphs + 1)
    tau_expectations = np.cumsum(tau_variable) / np.arange(1, n_graphs + 1)

    return xi_expectations, tau_expectations, theoretical_expectation


def make_plot(x_data, x_names, const_vals, const_names, plot_title, val_length):
    fig = go.Figure()

    for value, name in zip(x_data, x_names):
        fig.add_trace(
            go.Scatter(
                x=np.arange(1, val_length + 1),
                y=value,
                mode="lines",
                line=dict(width=3),
                name=name,
            )
        )

    for const_val, const_name in zip(const_vals, const_names):
        fig.add_trace(
            go.Scatter(
                x=[1, val_length],
                y=[const_val, const_val],
                mode="lines",
                line=dict(dash="dash", width=3),
                name=const_name,
            )
        )

    fig.update_layout(
        title=plot_title,
        xaxis_title="Number of graphs",
        yaxis_title="Value",
        legend=dict(x=0.90, y=1.0),
        template="plotly_dark",
    )

    fig.show()

In [32]:
def estimate_and_plot_expectation_convergence(n_graphs, n_nodes, edge_p):
    xi_expectations, tau_expectations, theoretical_expectation = (
        get_xi_and_tau_and_theoretical_expectations(n_graphs, n_nodes, edge_p)
    )
    print("Theoretical E_xi:", theoretical_expectation)
    print("M_k:", xi_expectations[-1])
    print("T_k:", tau_expectations[-1])

    if MAKE_PLOTS:
        make_plot(
            x_data=[xi_expectations, tau_expectations],
            x_names=["M(k)", "T(k)"],
            const_vals=[theoretical_expectation],
            const_names=["E_xi"],
            plot_title=f"Convergence of M(k) and T(k) to E_xi for G({n_nodes}, {edge_p})",
            val_length=k,
        )

In [33]:
np.random.seed(43)
k = 10**3
n = 10
p = 0.5

estimate_and_plot_expectation_convergence(k, n, p)

Theoretical E_xi: 15.0
M_k: 14.832
T_k: 14.979


![](plots/random_graph/E_convergence_G_10_05.png)

In [34]:
np.random.seed(41)
p = 0.05

estimate_and_plot_expectation_convergence(k, n, p)

Theoretical E_xi: 0.015000000000000003
M_k: 0.015
T_k: 0.014


![](plots/random_graph/E_convergence_G_10_005.png)

## Task 3

### It's easy to calculate the variance for one a graph on three vertices:
$E\tau_1 = 1 \cdot p^3 + 0 \cdot (1 - p^3) = p^3$<br>

$D\tau_1 = E\tau_1^2 - (E\tau_1)^2 = 1^2 \cdot p^3 - p^6 = p^3 - p^6$

And for $С_n^3$ generations: $D\tau = C_n^3 \cdot D\tau_1 = C_n^3 \cdot (p^3 - p^6);$ because each iteration is independent.
<br><br>

### For a random graph on n vertices:
$D \xi = E \xi^2 - (E \xi)^2$
<br>
$E \xi^2 = E \left( \sum_{i=1}^{C_n^3} I_i \right)^2 = 
\sum_{i=1}^{C_n^3} E I_i^2 + \sum_{i \neq j} E I_i I_j = 
C_n^3 \cdot p^3 + C_n^3 \cdot C_{n-3}^3 \cdot p^6 + 3 \cdot C_n^3 \cdot C_{n-3}^2 \cdot p^6 + 3 \cdot C_n^3 \cdot (n-3) \cdot p^5$
<br>
$(E \xi)^2 = (C_n^3 \cdot p^3)^2$

In [35]:
def get_theoretical_xi_and_tau_var(n_nodes, edge_p):
    tau_var = comb(n_nodes, 3) * (edge_p**3 - edge_p**6)

    xi_squared_E = (comb(n_nodes, 3) * edge_p**3) ** 2
    xi_second_moment = (
        comb(n_nodes, 3) * edge_p**3
        + comb(n_nodes, 3) * comb(n_nodes - 3, 3) * edge_p**6
        + 3 * comb(n_nodes, 3) * comb(n_nodes - 3, 2) * edge_p**6
        + 3 * comb(n_nodes, 3) * (n_nodes - 3) * edge_p**5
    )
    xi_var = xi_second_moment - xi_squared_E

    return xi_var, tau_var


def print_xi_and_tau_vars(k_graphs, n_nodes, edge_p):
    xi_variable = generate_xi(k_graphs, n_nodes, edge_p)
    tau_variable = generate_tau(k_graphs, n_nodes, edge_p)
    xi_var = np.var(xi_variable)
    tau_var = np.var(tau_variable)

    theoretical_xi_var, theoretical_tau_var = get_theoretical_xi_and_tau_var(
        n_nodes, edge_p
    )

    print(f"### G({n_nodes}, {edge_p}) ###\n")
    print(
        f"  Theoretical Variances: \nxi: {theoretical_xi_var} \ntau: {theoretical_tau_var}"
    )
    print(f"\n  Real Variances: \nxi: {xi_var} \ntau: {tau_var}")

In [36]:
k = 3 * 10**5
n = 10

In [37]:
p = 0.5
print_xi_and_tau_vars(k, n, p)

### G(10, 0.5) ###

  Theoretical Variances: 
xi: 52.5 
tau: 13.125

  Real Variances: 
xi: 52.395481053233325 
tau: 13.102612210566669


In [38]:
p = 0.05
print_xi_and_tau_vars(k, n, p)

### G(10, 0.05) ###

  Theoretical Variances: 
xi: 0.015746250000000003 
tau: 0.014998125000000005

  Real Variances: 
xi: 0.01573843332222222 
tau: 0.015090376488888887


#### And let's plot the convergence of the variances

In [39]:
def get_vars_for_plot(n_graphs, n_nodes, edge_p):
    xi_variable = generate_xi(n_graphs, n_nodes, edge_p)
    tau_variable = generate_tau(n_graphs, n_nodes, edge_p)

    xi_vars = [np.var(xi_variable[:i]) for i in range(2, k)]
    tau_vars = [np.var(tau_variable[:i]) for i in range(2, k)]

    return xi_vars, tau_vars


def estimate_and_plot_variance_convergence(k_graphs, n_nodes, edge_p):
    xi_vars, tau_vars = get_vars_for_plot(k_graphs, n_nodes, edge_p)
    theoretical_xi_var, theoretical_tau_var = get_theoretical_xi_and_tau_var(
        n_nodes, edge_p
    )

    if MAKE_PLOTS:
        make_plot(
            x_data=[xi_vars, tau_vars],
            x_names=["xi", "tau"],
            const_vals=[theoretical_xi_var, theoretical_tau_var],
            const_names=["Theoretical xi", "Theoretical tau"],
            plot_title=f"xi and tau Convergence of Variances for G({n_nodes}, {edge_p})",
            val_length=len(xi_vars),
        )

#### G(10, 0.5)

In [40]:
np.random.seed(13)

k = 10**4
n = 10
p = 0.5

estimate_and_plot_variance_convergence(k, n, p)

![](plots/random_graph/xi_and_tau_var_convergence_G_10_05.png)

#### And G(10, 0.05)

In [41]:
np.random.seed(42)
p = 0.05

estimate_and_plot_variance_convergence(k, n, p)

![](plots/random_graph/xi_and_tau_var_convergence_G_10_005.png)